__________________________________________________________________________________________________________

# EJEMPLO 1
Un proceso envia un numero, otro porceso lo recibe y lo recorre hasta cero usando hilos

__________________________________________________________________________________________________________

In [1]:
%pycat final1.cpp

#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>
#include<omp.h>

using namespace std;

int main(int argc, char* argv[]) {
 
    MPI_Init(&argc, &argv);

    int size, rank;
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    int numenv=20;
    int numrec;
    
    //Instrucciones para el proceso 1
    if(rank==1){
        
        //Recibe el numero el proceso 0 envie
        MPI_Recv(&numrec,1,MPI_INT,0,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
        
        //Mediante el numero seleccionado de hilos se reparten el trabajo de ir disminuyendo el numero recibido hasta 0
        #pragma omp parallel num_threads(5)
        {
            
            #pragma omp for
            for (int i=0 ; i<=numenv ; i++)
            {
                #pragma omp critical
                {
                    int hilonum = omp_get_thread_num();
                    printf("Proceso %d, hilo %d muestra el numero %d \n", rank, hilonum, numrec);
                  

In [2]:
! mpicxx -fopenmp final1.cpp -o final1.out

In [3]:
! mpirun -np 3 ./final1.out

El proceso 0 envia el numero 20 

Proceso 1, hilo 1 muestra el numero 20 
Proceso 1, hilo 1 muestra el numero 19 
Proceso 1, hilo 2 muestra el numero 18 
Proceso 1, hilo 2 muestra el numero 17 
Proceso 1, hilo 3 muestra el numero 16 
Proceso 1, hilo 3 muestra el numero 15 
Proceso 1, hilo 1 muestra el numero 14 
Proceso 1, hilo 3 muestra el numero 13 
Proceso 1, hilo 0 muestra el numero 12 
Proceso 1, hilo 3 muestra el numero 11 
Proceso 1, hilo 1 muestra el numero 10 
Proceso 1, hilo 0 muestra el numero 9 
Proceso 1, hilo 2 muestra el numero 8 
Proceso 1, hilo 4 muestra el numero 7 
Proceso 1, hilo 0 muestra el numero 6 
Proceso 1, hilo 2 muestra el numero 5 
Proceso 1, hilo 0 muestra el numero 4 
Proceso 1, hilo 4 muestra el numero 3 
Proceso 1, hilo 0 muestra el numero 2 
Proceso 1, hilo 4 muestra el numero 1 
Proceso 1, hilo 4 muestra el numero 0 


__________________________________________________________________________________________________________

# EJEMPLO 2
Se suman los valores de un array mediante varios procesos, primero cada proceso hace una suma parcial y al final se realiza la suma total

__________________________________________________________________________________________________________

In [4]:
%pycat final2.cpp

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <mpi.h>

using namespace std;

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv);

    int world_size, world_rank;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);

    const int array_size = 100;
    int array[array_size];

    // Inicializa el array con valores aleatorios 
    if (world_rank == 0) {
        srand(time(NULL));
        for (int i = 0; i < array_size; ++i) {
            array[i] = rand() % 100;
            //array[i] = 5;
        }
    }

    // Distribuiye el array entre los procesos MPI
    MPI_Bcast(array, array_size, MPI_INT, 0, MPI_COMM_WORLD);

    // Cada proceso MPI suma su parte del array
    int local_sum = 0;
    for (int i = 0; i < array_size / world_size; ++i) {
        local_sum += array[i + world_rank * (array_size / world_size)];
    }

    // Imprime el trabajo que hace cada proceso en paralelo
    cout << "( Proceso " << world_ran

In [5]:
! mpicxx final2.cpp -o final2.out

In [6]:
! mpirun -np 5 ./final2.out

( Proceso 0 Suma parcial: 952 ) 
( Proceso 1 Suma parcial: 960 ) 
( Proceso 2 Suma parcial: 1074 ) 
( Proceso 3 Suma parcial: 1148 ) 
( Proceso 4 Suma parcial: 975 ) 
Suma Total: 5109
